# Exp 05: Demonstrate Numba
Let's build upon the 


In [2]:
# standard libraries
from time import perf_counter_ns
import time

In [3]:
# external libraries
from matplotlib.patches import Rectangle
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import seaborn as sns

# custom libraries
from _run_constants import *
from part_00_file_db_utils import *
from part_00_process_functions import *

In [4]:
word_df, wg_df, letter_dict, char_matrix, \
    word_group_id_list, word_id_list, wchar_matrix = load_input_data(
        db_path=rc.DB_PATH, db_name=rc.DB_NAME,
        in_file_path=rc.IN_FILE_PATH)

...loading words into a dataframe...
...query execution took: 1.34 seconds...
...loading word groups into a dataframe...
...query execution took: 1.28 seconds...
...loading the letter dictionary...
...loading the char matrix...
...subsetting the char matrix...


In [13]:
ls_df = build_letter_selector_df(df = wg_df, 
                                 ls_nchar=3, letter_selector_col_name='letter_selector',
                                 letter_selector_id_col_name='letter_selector_id')

In [14]:
# function to return the index position of each letter
def get_ls_index(ls:str):
    return [letter_dict[l] for l in ls]

In [15]:
ls_df['ls_index'] = ls_df['letter_selector'].map(get_ls_index)

In [16]:
ls_df.head()

,letter_selector,ls_count,ls_nchar_iter,ls_nchar,letter_selector_id,ls_index
0,a,2,3,1,0,[0]
1,ae,1,3,2,1,"[0, 4]"
2,ai,1,3,2,2,"[0, 8]"
3,b,1,3,1,3,[1]
4,ba,4,3,2,4,"[1, 0]"


In [9]:
# load the total number of anagrams
n_possible_anagrams = load_possible_anagrams(db_path=rc.DB_PATH,
                                             db_name=rc.DB_NAME)

...query execution took: 0.0 seconds...


In [20]:
wg_df.head()

,word,lcase,n_chars,first_letter,word_id,word_group_id,letter_group,letter_group_ranked,word_group_count,letter_selector,n_records,letter_selector_id
0,A,a,1,a,0,0,a,a,1,a,1,0
1,aa,aa,2,a,1,1,a,a,1,a,1,0
2,aal,aal,3,a,2,2,al,la,2,la,1,1081
3,aalii,aalii,5,a,3,3,ail,lai,1,lai,1,1083
4,aam,aam,3,a,4,4,am,ma,2,ma,1,1114


In [19]:
col_names = ['letter_selector', 'letter_selector_id']
wg_df = pd.merge(left = wg_df, right = ls_df[col_names])

In [21]:
#curr_wg_id_list = wg_df.loc[wg_df['letter_selector_id'] == ls_id, 'word_group_id'].to_numpy(dtype = np.int32)
test_matrix = wg_df[['letter_selector_id', 'word_group_id']].to_numpy()

In [22]:
from numba import njit, int8, int32

In [23]:
#temp_wg_id_list[np.all(a = (ls_wchar_matrix - ls_wchar_matrix[temp_wg_id, :]) >= 0, axis = 1)]
#@njit
# def build_output_wg_id_list(temp_wg_id_list:np.ndarray, ls_wchar_matrix:np.ndarray, temp_wg_id:int):
#     temp_matrix = ls_wchar_matrix - ls_wchar_matrix[temp_wg_id, :]    
#     zero_list = np.zeros(shape = temp_matrix.shape[0], dtype=np.int8)
#     for i in range(temp_matrix.shape[0]):
#         zero_list[i] = np.min(temp_matrix[i, :]) >= 0
#     zero_list = zero_list.astype(np.bool)
#     return temp_wg_id_list[zero_list]
#@njit(int32[:](int32[:], int8[:,:], int32))
@njit()
def build_output_wg_id_list(temp_wg_id_list:np.ndarray, ls_wchar_matrix:np.ndarray, temp_wg_id:int) -> np.ndarray:
    rows, cols = ls_wchar_matrix.shape
    temp_matrix = (ls_wchar_matrix - ls_wchar_matrix[temp_wg_id, :]) >= 0    
    zero_list = np.zeros(shape = rows, dtype=np.bool)
    for i in range(rows):
        zero_list[i] = temp_matrix[i, :].sum() == cols
    #zero_list = zero_list.astype(np.bool)
    return temp_wg_id_list[zero_list]

In [24]:
# run it!
run_start_time=perf_counter_ns()
# create the output list
output_list = np.full(shape = (n_possible_anagrams, 2), fill_value=-1)
output_time_list = []

# start counting
anagram_pair_count = 0

for ls_row_id, ls_row in ls_df.iloc[:None].iterrows():    
    if ls_row_id % 100 == 0:
        print(ls_row_id)
    start_time = perf_counter_ns()
    
    # get letter selector id information    
    ls_id = ls_row['letter_selector_id']
    ls_id_index = np.array(object  = ls_row['ls_index'])    
        
    ##
    # SUBSET THE wchar_matrix by column selector
    ##    
    outcome_indices = np.all(wchar_matrix[:, ls_id_index] >= 1, axis=1)
    
    # this is the sub-matrix from which to query
    ls_wchar_matrix = wchar_matrix[outcome_indices, :]
        
    # this is the list of word group ids that correspond to the word group ids
    # in the ls_wchar_matrix
    temp_wg_id_list = word_group_id_list[outcome_indices]
    # place into a dictionary to go from wg_id to wg_index. What is the index
    # of wg_id 675?
    # wg_id_dict = {wg_id:wg_index for wg_index, wg_id in enumerate(temp_wg_id_list)}

    # this is the number of word groups that meet certain criteria. 
    # for example, words that feature the letters: 'bro'    
    n_search_space = temp_wg_id_list.shape[0]
        
    #def my_func(row):
    #    return temp_wg_id_list[np.all(a = (ls_wchar_matrix - ls_wchar_matrix[row, :]) >= 0, axis = 1)]

    #for ii in range(0, ls_wchar_matrix.shape[0]):    
    #for i_curr_wg_id, curr_wg_id in enumerate(temp_wg_id_list):
    # the current list of words featuring the set of least common letters.
    # these are the words have the least common letters of 'bro'    
    #curr_wg_id_list = wg_df.loc[wg_df['letter_selector_id'] == ls_id, 'word_group_id'].to_numpy(dtype = np.int32)
    curr_wg_id_list = test_matrix[test_matrix[:, 0] == ls_id, 1]
    # n_lookups = curr_wg_id_list.shape[0]
    # n_search_space >= n_lookups, always. 
    for i_curr_wg_id, curr_wg_id in enumerate(curr_wg_id_list):    
            
        # get different word group ids?
        temp_wg_id = np.where(temp_wg_id_list == curr_wg_id)[0][0]
        #print(curr_wg_id, temp_wg_id)

        #outcome_word_id_list = my_func(row = temp_wg_id)
        #outcome_word_id_list = temp_wg_id_list[np.all(a = (ls_wchar_matrix - ls_wchar_matrix[temp_wg_id, :]) >= 0, axis = 1)]
        outcome_word_id_list = build_output_wg_id_list(temp_wg_id_list=temp_wg_id_list,
                                                       ls_wchar_matrix=ls_wchar_matrix,
                                                       temp_wg_id=temp_wg_id)
                
        n_from_words = outcome_word_id_list.shape[0]
        
        if n_from_words > 0:
            outcome_word_id_list = format_output_list(outcome_word_id_list=outcome_word_id_list, wg_id=curr_wg_id)
            #print(outcome_word_id_list.shape)
            
            # enumerate the from/parent words
            new_anagram_pair_count = anagram_pair_count + n_from_words
            #print(anagram_pair_count, new_anagram_pair_count)

            output_list[anagram_pair_count:new_anagram_pair_count, :] = outcome_word_id_list

            # update the anagram pair count
            anagram_pair_count = new_anagram_pair_count

    curr_time = calc_time(time_start=start_time, round_digits=8)
    output_time_list.append([ls_id, n_search_space, curr_time])

print('...time to find parent/child word relationships')
time_proc = calc_time(time_start=run_start_time, round_digits=4)
compute_elapsed_time(seconds=time_proc)
print('...truncating output list...')
output_indices = np.all(output_list >= 0, axis=1)
output_list = output_list[output_indices,]
print(output_list.shape)
time_proc = calc_time(time_start=run_start_time, round_digits=4)
compute_elapsed_time(seconds=time_proc)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
...time to find parent/child word relationships
Hours: 0 | minutes: 2 | seconds: 28.9428
...truncating output list...
(73218235, 2)
Hours: 0 | minutes: 2 | seconds: 40.4498


In [25]:
# approximately 19 seconds
from_word_counter = collections.Counter(output_list[:,1])
to_word_counter = collections.Counter(output_list[:,0])

In [26]:
# the number of from word groups: should be 26
print(from_word_counter[746])
print(to_word_counter[746])

26
329


In [ ]:
# using numba: 2 minutes, 29 seconds. 